<a href="https://colab.research.google.com/github/crux007/crux007/blob/main/refined_ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

Upload the dataset from my google drive

In [15]:
data = pd.read_csv("/content/drive/MyDrive/Data_Scaled.csv")

View the top rows of my dataset so that i can see the column headers

In [16]:
data.head()

,MonthlyRate,EnvironmentSatisfaction,Age,TotalWorkingYears,PercentSalaryHike,YearsInCurrentRole,DistanceFromHome,HourlyRate,JobSatisfaction,PerformanceRating,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,Attrition
0,0.718456,-0.435052,0.614919,-0.266327,-1.148653,0.111200,-1.123514,1.524297,1.397211,-0.341554,...,0,0,1,0,0,0,1,0,1,1
1,1.486362,0.496146,1.512244,0.002528,2.354025,1.004156,-0.210100,-0.225833,-0.460303,2.927798,...,0,1,0,0,0,1,0,1,0,0
2,-1.698297,1.427345,0.166256,-0.400754,0.018906,-1.079409,-0.993026,1.418229,0.468454,-0.341554,...,0,0,0,0,0,0,1,0,1,1
3,1.239070,1.427345,-0.282406,-0.266327,-1.148653,1.004156,-0.862538,-0.491005,0.468454,-0.341554,...,0,1,0,0,0,1,0,0,1,0
4,0.315688,-1.366251,-0.955400,-0.535182,-0.856763,-0.484104,-0.993026,-1.339553,-0.460303,-0.341554,...,0,0,0,0,0,1,0,1,0,0


split my dataset. using 80% for training and 20% for testing

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("Attrition", axis=1), data["Attrition"], test_size=0.2, random_state=42)

Instantiate K-Nearest Neighbour, Support Vector Machine, Random forest and gradient boosting

In [18]:
knn = KNeighborsClassifier()
svm = SVC(probability=True)
rf = RandomForestClassifier(n_estimators=100)
gbm = GradientBoostingClassifier()

In [19]:
rf_param_grid = {'n_estimators': [50, 100, 200],
                 'max_depth': [5, 10, None]}
svm_param_grid = {'C': [0.1, 1, 10],
                  'kernel': ['linear', 'rbf', 'poly']}

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
rf = RandomForestClassifier()

In [23]:
svm = SVC(probability=True)

In [24]:
rf_grid = GridSearchCV(rf, rf_param_grid, cv=5)
svm_grid = GridSearchCV(svm, svm_param_grid, cv=5)

In [25]:
knn_params = {'n_neighbors': [3, 5, 7], 'metric': ['manhattan']}

In [27]:
from sklearn.model_selection import cross_val_score

In [28]:
gbm_cv = cross_val_score(gbm, X_train, y_train, cv=5)

In [29]:
knn_gs = GridSearchCV(knn, knn_params, cv=5)
knn_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['manhattan'], 'n_neighbors': [3, 5, 7]})

Fit these 4 models on the training data

In [30]:
knn.fit(X_train, y_train)
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
gbm.fit(X_train, y_train)

GradientBoostingClassifier()

Make prediction on the performance of our model using the test data

In [31]:
knn_pred = knn.predict(X_test)
svm_pred = svm.predict(X_test)
rf_pred = rf.predict(X_test)
gbm_pred = gbm.predict(X_test)

Compute the accuracy for each model

In [32]:
knn_acc = accuracy_score(y_test, knn_pred)
svm_acc = accuracy_score(y_test, svm_pred)
rf_acc = accuracy_score(y_test, rf_pred)
gbm_acc = accuracy_score(y_test, gbm_pred)

Print result for each model

In [33]:
print("KNN Accuracy:", knn_acc)
print("SVM Accuracy:", svm_acc)
print("Random Forest Accuracy:", rf_acc)
print("Gradient Boosting Accuracy:", gbm_acc)

KNN Accuracy: 0.8789144050104384
SVM Accuracy: 0.9206680584551148
Random Forest Accuracy: 0.9373695198329853
Gradient Boosting Accuracy: 0.9227557411273486


combine all the model performances by creating an ensemble

In [34]:
ensemble = VotingClassifier(estimators=[('knn', knn), ('svm', svm), ('rf', rf), ('gbm', gbm)], voting='soft')

fit the ensemble model

In [35]:
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('knn', KNeighborsClassifier()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier()),
                             ('gbm', GradientBoostingClassifier())],
                 voting='soft')

Make predictions on the test data using the ensemble model

In [36]:
ensemble_pred = ensemble.predict(X_test)

Calculate the accuracy of the ensemble model

In [37]:
ensemble_acc = accuracy_score(y_test, ensemble_pred)

In [38]:
print("Ensemble Accuracy:", ensemble_acc)

Ensemble Accuracy: 0.941544885177453
